In [1]:
import openmmtools
from simtk.openmm import app
import simtk.openmm as mm
from simtk import unit
from sys import stdout
import numpy as np
import yank
from yank.restraints import RestraintState
from yank.experiment import Topography
from importlib import reload

In [2]:
pdb = app.PDBFile('tip3p.pdb')
forcefield = app.ForceField('amber99sbildn.xml', 'tip3p.xml')

system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.PME, 
    nonbondedCutoff=1.0*unit.nanometers, constraints=app.HBonds, rigidWater=True, 
    ewaldErrorTolerance=0.0005)

integrator = mm.LangevinIntegrator(100*unit.kelvin, 1.0/unit.picoseconds, 
    2.0*unit.femtoseconds)
integrator.setConstraintTolerance(0.00001)

platform = mm.Platform.getPlatformByName('CUDA')
properties = {'CudaPrecision': 'mixed'}

In [4]:
reload(openmmtools.alchemy)
factory = openmmtools.alchemy.AbsoluteAlchemicalFactory(consistent_exceptions=False, split_alchemical_forces = True)#, alchemical_pme_treatment = 'exact')
reference_system = system
alchemical_region_1 = openmmtools.alchemy.AlchemicalRegion(alchemical_atoms = [3,4,5], name='one')
alchemical_region_0 = openmmtools.alchemy.AlchemicalRegion(alchemical_atoms = [0,1,2], name='zero')
alchemical_region_2 = openmmtools.alchemy.AlchemicalRegion(alchemical_atoms = [6,7,8], name='two')
alchemical_region_3 = openmmtools.alchemy.AlchemicalRegion(alchemical_atoms = [9,10,11], name='three')

alchemical_system_in = factory.create_alchemical_system(reference_system, alchemical_regions = [alchemical_region_0, alchemical_region_1, alchemical_region_2, alchemical_region_3])

alchemical_state_zero = openmmtools.alchemy.AlchemicalState.from_system(alchemical_system_in, region_name = 'zero')
alchemical_state_one = openmmtools.alchemy.AlchemicalState.from_system(alchemical_system_in, region_name = 'one')
alchemical_state_two = openmmtools.alchemy.AlchemicalState.from_system(alchemical_system_in, region_name = 'two')
alchemical_state_three = openmmtools.alchemy.AlchemicalState.from_system(alchemical_system_in, region_name = 'three')


Using 4 alchemical regions


In [4]:
#restraint_state = RestraintState(lambda_restraints=1.0)
#print(restraint_state)

In [5]:
TS = openmmtools.states.ThermodynamicState(alchemical_system_in, temperature=300*unit.kelvin, pressure=1*unit.bar)
composable_states = [alchemical_state_zero, alchemical_state_one, alchemical_state_two, alchemical_state_three]#, restraint_state]
compound_state = openmmtools.states.CompoundThermodynamicState(
            thermodynamic_state=TS, composable_states=composable_states)

In [8]:
reload(openmmtools.alchemy)
integrator = mm.VerletIntegrator(1.0*unit.femtosecond)
compound_state._composable_states[0].lambda_sterics_zero = 1.0
compound_state._composable_states[0].lambda_electrostatics_zero = 1.0
compound_state._composable_states[1].lambda_sterics_one = 0.0
compound_state._composable_states[1].lambda_electrostatics_one = 0.0
compound_state._composable_states[2].lambda_sterics_two = 1.0
compound_state._composable_states[2].lambda_electrostatics_two = 1.0
compound_state._composable_states[3].lambda_sterics_three = 1.0
compound_state._composable_states[3].lambda_electrostatics_three = 1.0
context = compound_state.create_context(integrator)

In [9]:
context.setPositions(pdb.positions)
#for param in context.getParameters():
    #print(param)
    #print(context.getParameter(param))

integrator.step(1000)
print('No Error')

No Error
